## Quantization in Large Language Models

### **Introduction to Quantization**

Quantization is a process used to reduce the memory requirements and computational complexity of large machine learning models. By representing model parameters with lower-precision values, quantization makes it possible to run models more efficiently on devices with limited memory and computational resources.

For large language models (LLMs), quantization can:
- **Reduce Memory Usage:** Lower-precision data types (such as int8) use less memory than higher-precision types (like float32), allowing models to fit into memory-constrained environments.
- **Improve Inference Speed:** By using simpler operations on smaller data types, quantization can reduce the time it takes for a model to process inputs and generate outputs.
- **Preserve Accuracy:** Quantization is carefully designed to minimize the impact on model accuracy, though a trade-off often exists between precision and efficiency.

We will focus on Post-Training Quantization (PTQ), a quantization technique that applies quantization to a pre-trained model. PTQ is a popular method for quantizing large language models because it can be applied to a wide range of models that we may want to use in inference mode. By contrast, quantization-aware training (QAT) requires retraining the model with quantization in mind, which can be more complex and time-consuming.

We will first get a general understanding of quantization by manually implementing two commonly adopted approaches: absmax and minmax (or zero-point). 

Next, we will explore two different ways (one using PyTorch, the other using HuggingFace) to run a **dynamic quantization** (i.e., PTQ where only the weights are quantized, and not the activations). 

## 1. Absmax and Minmax Quantization

The goal of quantization is, remember, mapping continuos values (e.g., float32) into a discrete set of values (e.g., int8). 

So let's create a matrix $W$ and a vector $x$ to be quantized. Let's initialize them randomly (but, just to see what happens, let's set W[0,0] and x[0] = 0).

In [ ]:
import torch

torch.random.manual_seed(0)

n_rows = 3
n_cols = 5
W = torch.randn(n_rows, n_cols)
x = torch.randn(n_cols)

W[0,0] = 0
x[0] = 0

In [ ]:
W.shape, x.shape

Let's first compute the matrix multiplication to observe the result. This is the operation that we typically want to execute, and that we want to quantize.

We will quantize $W$ and $x$ separately, and then multiply them together. Finally, we will need to dequantize the result to compare it with the original result.

In [ ]:
out = W @ x
print(out)

## 1.1 Absmax quantization

In absmax quantization, we use a symmetric range around 0. This means that we need to identify the maximum absolute value in the matrix $W$ and the vector $x$.

We define a function `absmax_quantize` that takes as input any tensor and produces a version of the same tensor, but quantized. 

In [ ]:
def absmax_quantize(W):
    # NOTE: we assume that we always map to 8-bit integers
    max_value = W.abs().max()
    scale = 127 / max_value # how "long" the step between any two int8 values is
    W_q = (W * scale).round().to(torch.int8)
    return W_q, scale

Notice, we return both the quantized tensor and the scale factor. The scale factor is used to dequantize the tensor. So we might as well define a dequantize function:

In [ ]:
def absmax_dequantize(W_q, scale):
    return W_q.float() / scale

Let's get the quantized version of W, and of x. Then, we can check how much we are losing by quantizing the values.

In [ ]:
W_q, scale_W = absmax_quantize(W)
x_q, scale_x = absmax_quantize(x)

In [ ]:
# print(W_q)
print(W)
W_deq = absmax_dequantize(W_q, scale_W)
print(W_deq)
print((W - W_deq).abs().mean())

In [ ]:
print(x_q)
print(x)
x_deq = absmax_dequantize(x_q, scale_x)
print(x_deq)
print((x - x_deq).abs().mean())

Notice that, in both cases, absmax maps the value 0 to 0. This is a good property, as it allows us to represent the zero value without losing any information. This property stems from the symmetry around 0 we imposed.

However, do note that we are also "wasting" some bits of the range! Can you spot where?

Let's now compute the matrix multiplication between W_q and x_q. 

In [ ]:
W_q @ x_q

Can you see that there's something wrong? Let's see what one of the rows of W_q and x_q contain:

In [ ]:
W_q[0], x_q

In [ ]:
a = torch.tensor(127, dtype=torch.int8)
b = torch.tensor(1, dtype=torch.int8)
c = a + b   
print(c) 

The dot product of these two vectors definitely isn't what we get as the first number of the matrix multiplication -- i.e. (W_q @ x_q)[0]. Indeed, we can run as int16, and see that the result is quite different:

In [ ]:
W_q.to(torch.int16) @ x_q.to(torch.int16)

The result of the dot product overflows the int8 range. This is a well-known problem. Indeed, the accumulation of results, in quantization, is typically done with higher precision than the single values. This is tricky to do in pure Python/PyTorch, but can be done efficiently in other ways.

Let's stick to the simple approach for now. 

In [ ]:
out_q = W_q.to(torch.int16) @ x_q.to(torch.int16)

To get the correct result, we need to dequantize the result. This is done by multiplying the result by the scale factor of the two operands.

In [ ]:
out_deq = absmax_dequantize(absmax_dequantize(out_q, scale_W), scale_x)
# Or, alternatively:
# out_deq = out_q / scale_W / scale_x
# out_deq = absmax_dequantize(out_q, scale_W * scale_x)
print(out_deq)
print(out)

Remember, our goal was `out`. How much did we lose by quantizing and dequantizing?

In [ ]:
(out_deq - out).abs().mean()

## 2. Minmax Quantization

In minmax quantization, we use the minimum and maximum values in the matrix $W$ and the vector $x$ to define the range. In this way, we get a range that is as tight as possible around the values we are quantizing. This will, however, change the zero value, which will not be mapped to 0 anymore.



In [ ]:
def minmax_quantize(W):
    # the following notations come from:
    # (1) scaling W to [0,1] ==> W' = (W - min(W)) / (max(W) - min(W)),
    # (2) scaling W' to [-128, 127] ==> W_q = W' * 255 - 128
    # by combining the two, we get that:
    # W_q = W * scale + offset
    # we will call the offset "zero_point", as it represent the value that maps to 0
    delta = W.max() - W.min()
    scale = 255 / delta
    zero_point = -(128*W.max() + 127*W.min()) / delta
    W_q = (W * scale + zero_point).round().to(torch.int8)
    return W_q, scale, zero_point

def minmax_dequantize(W_q, scale, zero_point):
    return (W_q.float() - zero_point) / scale

In [ ]:
W_q, scale_W, zero_point_W = minmax_quantize(W)
x_q, scale_x, zero_point_x = minmax_quantize(x)

Let's see the results for $W$ (same considerations will apply for $x$).

In [ ]:
print(W_q)
print(W)
W_deq = minmax_dequantize(W_q, scale_W, zero_point_W)
print(W_deq)
print((W - W_deq).abs().mean())

First, notice that 0 no longer maps to 0! Indeed, it maps to zero_point_W (after rounding). This implies that the dequantization of 0 will no longer be 0. This may be a problem!

But, notice that we are using the full range of the int8 values. This means that we are not wasting any bits of the range! (the minimum value is -128, the maximum value is 127). This can also be seen in the average absolute error, which is lower than what we had with absmax.

Similarly to what we did before, let's compute the output of the operation, and then dequantize it!

In [ ]:
out_q = W_q.to(torch.int16) @ x_q.to(torch.int16)
print(out_q)

The dequantification is a bit trickier, in this case. Can you figure out why we need the following operations?

Hint: consider the transformation we are applying to each value (value * scale + zero_point). What happens when we compute the dot product?

In [ ]:
out_deq = (out_q - W.shape[1] * zero_point_W * zero_point_x - W.sum(axis=1) * scale_W * zero_point_x - x.sum() * scale_x * zero_point_W) / (scale_W * scale_x)

In [ ]:
print(out_deq)
print(out)
print((out_deq - out).abs().mean())

<span style="color:red">Extra stuff!</span>

We could have computed scales and zero points at different granularities (e.g., for each row, or column of $W$). How would that have changed the results? What changes would we have to do to the code?

# Dynamic quantization

In this second part, we will apply dynamic quantization by using PyTorch or HuggingFace (with BitsAndBytes). We will quantize both to 8 and to 4 bits, and we will see how that affects LLMs (in terms of memory and speed). 

In [ ]:
import torch
import os
import time 
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from huggingface_hub import login

# Login to the Hugging Face model hub to be able to upload models
with open("../hf_token.txt", "r") as f:
    token = f.read()
    f.close()

login(token=token)

First, let's load our model (Llama 3.2 1B) and let's see some base statistics (memory usage, inference time).

In [ ]:
model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id)  
tokenizer = AutoTokenizer.from_pretrained(model_id) 

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def get_model_size(model):
    """Get the size of the model in MB"""
    torch.save(model.state_dict(), "temp.pth")
    size = os.path.getsize("temp.pth") / 1e6  # size in "MB" (technically, it should be 1024**2, but we approximate to 1e6 to get an easier conversion #params <=> MB)
    os.remove("temp.pth")
    return size

print(f"Model size before quantization {(get_model_size(model)):.2f} MB")

Wait, wasn't Llama 1B supposed to be 4GB (4 bytes * 1B parameters)? Why do we get ~ 5 GB (i.e., 1.25B parameters)?

We are not considering the parameters used in the embedding layer (you can count how many parameters you have in the embedding layer and see that it matches the difference). 

Additionally, the count does not include the `lm_head`, i.e. the layer used to go from the hidden states to the logits. This is because in Llama (and other models) the `lm_head` is shared with the embedding layer, so it is not counted twice.

In [ ]:
text = "The secret of life is"
# Notice we use a batch of 20 sentences -- we will get better results
# on quantized models when processing a batch of inputs
inputs = tokenizer([text]*20, return_tensors="pt")

tic = time.time()

with torch.no_grad():
    baseline_output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

baseline_decoded = tokenizer.decode(baseline_output[0], skip_special_tokens=True)

print("\nBaseline model output:", baseline_decoded)
print("\nTime taken for baseline model:", elapsed_time)

Dynamic quantization applies lower precision to model weights and activations at runtime. This method doesn’t require modifications to the model architecture or retraining, which makes it relatively easy to apply.

- **Advantages:** 
  - Quick to implement with minimal changes. No calibration step is needed.

- **Limitations:** 
  - Activations are not pre-quantized, meaning some precision is maintained but at the cost of slightly higher resource use at inference time.


We use **[TorchAO](https://docs.pytorch.org/ao/stable/api_ref_quantization.html?utm_source=chatgpt.com)** to apply dynamic quantization to a model. TorchAO is the new quantization framework that replaces the deprecated `torch.ao.quantization` APIs.  

We can specify a set of layer types to be quantize. Let's stick with Linear layers. We specify the desired type, and off we go!

In [ ]:
import torch
from torchao.quantization import quantize_, Int8DynamicActivationInt8WeightConfig

quantize_(
    model,
    Int8DynamicActivationInt8WeightConfig(),
    filter_fn=lambda m, name: isinstance(m, torch.nn.Linear),
    device="cpu",
)

print(f"Model size after quantization {get_model_size(model):.2f} MB")

Okay -- 2.3GB? Why not 5GB / 4 = 1.25GB? After all, we are going from float32 to int8. 

That's correct -- technically. Except, we are only encoding linear layers, and not the embedding layer. That means that, of the original 1.25B parameters, we are only quantizing 1B. The rest, in the embedding layer, is kept as float32.

If you run the numbers, though, you should still find a problem: 1B * 1 byte + 0.25B * 4 bytes = 2GB. What about the rest? There's one more thing: remember, the `lm_head` was shared with the Embedding layer. However, since it is "copied" into a linear layer in Llama, the quantization process will quantize it as well. So that's an extra 0.25B parameters encoded as int8 -- hence 2.3GB.

Finally, we could technically also quantize the embeddings (it has been introduced in later versions of PyTorch), but for simplicity we will not do it here (it would require some additional steps).

In [ ]:
model

In [ ]:
tic = time.time()

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

output_decoded = tokenizer.decode(baseline_output[0], skip_special_tokens=True)

print("\nQuantized model output:", output_decoded)
print("\nTime taken for baseline model:", elapsed_time)

Hugging Face provides several built-in quantization options, each suited to different model and deployment needs:
https://huggingface.co/docs/transformers/v4.46.0/quantization/overview

For this lab, we will use `Quanto`.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
import torch
    
model_id = "meta-llama/Llama-3.2-1B"

# Quantize to 8-bit weights
quant = QuantoConfig(weights="int8")

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant,
    device_map="cpu",      
)

print(f"Model size after quantization: {get_model_size(model)} MB")

In [ ]:
model.model.layers[0].self_attn.q_proj.weight

In [ ]:
import time 

text = "The secret of life is"
inputs = tokenizer([text]*20, return_tensors="pt")

tic = time.time()

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

output_decoded = tokenizer.decode(baseline_output[0], skip_special_tokens=True)

print("\nquantized model output:", output_decoded)
print("\nTime taken for baseline model:", elapsed_time)